<a href="https://colab.research.google.com/github/nephelim74/spark/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled15_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Домашнее задание 3
Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.
Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)
Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта. Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, to_date
from pyspark.sql import functions as F


In [ ]:
# Создаем DataFrame
spark = SparkSession.builder.appName("Sales Data Analysis").getOrCreate()

In [ ]:
# Данные о продажах
data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]

columns = ["date", "category", "quantity", "revenue"]

# Создаем DataFrame
df = spark.createDataFrame(data, schema=columns)
df.show()


+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [ ]:
# Преобразуем столбец date в тип date
df = df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

In [ ]:
# Рассчитываем среднее значение выручки по категории и дате
avg_revenue_df = df.groupBy("date", "category").agg(F.avg("revenue").alias("avg_revenue")).orderBy("date", "avg_revenue")
# avg_revenue_df = avg_revenue_df.orderBy("date")
avg_revenue_df.show()


+----------+-----------+-----------+
|      date|   category|avg_revenue|
+----------+-----------+-----------+
|2023-11-20|      Books|     9000.0|
|2023-11-20|Electronics|    12000.0|
|2023-11-20|   Clothing|    15000.0|
|2023-11-21|      Books|    12000.0|
|2023-11-21|Electronics|    13000.0|
|2023-11-21|   Clothing|    14000.0|
|2023-11-22|      Books|    10000.0|
|2023-11-22|Electronics|    12500.0|
|2023-11-22|   Clothing|    16000.0|
+----------+-----------+-----------+



In [ ]:
# Применяем pivot для преобразования данных
pivot_df = avg_revenue_df.groupBy("category").pivot("date").agg(F.first("avg_revenue"))


In [ ]:
# Выводим результат
pivot_df.show()


+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [ ]:
# Останавливаем SparkSession
spark.stop()